In [42]:
!python --version

Python 3.5.2


In [19]:
from dcicutils import ff_utils
from functions.notebook_functions import *
import xlrd
import xlwt
import json

my_key = get_key('koray_data')
pr = ff_utils.get_metadata('/profiles/', my_key)

all_types = {}
for i in pr:
    if i.startswith('Exp'):
#         if i != 'ExperimentMic':
#             continue
        if 'experiment_type' in pr[i]['properties']:
            types = pr[i]['properties']['experiment_type']['enum']
            all_types[i] = types
            
all_t = []
for k,v in all_types.items():
    all_t.extend(v)
print(', '.join(all_t))


DAM-ID seq, DNA-paint, DNA FISH, RNA FISH, FISH, Immunofluorescence, Fluorescence, Cryo-EM, SPT, ChIA-PET, PLAC-seq, Hi-ChIP, ATAC-seq, Repli-seq, single cell Repli-seq, TSA-seq, capture Hi-C, ChIP-seq, RNA-seq, NAD-seq, DNA SPRITE, RNA-DNA SPRITE, MARGI, GAM, CUT&RUN, TrAC-loop, TRIP, in situ Hi-C, dilution Hi-C, micro-C, DNase Hi-C, TCC, cryomilling TCC, single cell Hi-C, sci-Hi-C, MC-3C, MC-Hi-C


In [3]:
### PLEASE COPY NOTEBOOKS TO YOUR FOLDERS TO PREVENT COMMIT CONFLICTS

from dcicutils import ff_utils
from functions.notebook_functions import *
import xlrd
import xlwt
import json

# get key from keypairs.json
my_key = get_key('koray_data')
schema_name = get_schema_names(my_key) 

# template excel
excel_file = '/Users/koray/Desktop/Wrangling/Labs/Gregor Lab/160515_spt_template.xls'

# summary excel
summary_file = '/Users/koray/Desktop/Wrangling/Labs/Gregor Lab/Gregor_lab_mRNA_FISH.xlsx'

#alias pre text
submitter_lab = 'thomas-gregor-lab:'

#if pre assign biosource define bio_source else set to false
bio_source = ''
bio_source = False
add_bcc = False
#project_name
project = 'BAF180'

In [4]:
import pandas as pd
# turn excel to pandas dataframe
df_sum= pd.read_excel(summary_file, index_col=None)
# create a new index
df_sum.reset_index(inplace=True)
# convert nan to None
df_sum = df_sum.where((pd.notnull(df_sum)), None)
# convert pandas dataframe to python list of dict
sum_list = df_sum.to_dict(orient='records')
print(len(sum_list), "lines on summary file")
print(sum_list[0])

153 lines on summary file
{'Set Description': "Single molecule mrnaFISH on endogenous hunchback using dual color 5' and 3' block probe sets", 'Cell Line': 'oreR', 'Number of Processed Files': 1.0, 'index': None, 'Experiment Sets (alias root)': 'hunchback_2colorBlock', 'Number of Raw Files': 1.0, 'Biosample (alias root)': '160216_oreR_hb2colorBlock_01', 'Experiment (alias root)': '160216_oreR_hb2colorBlock_01'}


In [7]:
TEMPO = {'imaging_path': [], 
         'biosource' : [], 
        'protocol': [], 
        'file_processed': [],
        'file_microscopy': [],
        'experiment_mic': [], 
        'microscope_setting_a2': [], 
#         'biosample_cell_culture': [], 
        'biosample': [], 
        'experiment_set_replicate': [],
        'publication':[]}

# Generate settings that is common to all
setting_alias = submitter_lab + 'mic_set_' + project
TEMPO['microscope_setting_a2'].append({'aliases':[setting_alias], 'tier_level': 'A2'})
print(TEMPO)

# Generate 2 protocols
experiment_protocol = submitter_lab + 'protocol_experiment_' + project

biosample_protocol = submitter_lab + 'protocol_biosample_' + project

TEMPO['protocol'].extend(
    [
    {'aliases':[experiment_protocol], 
     "protocol_type": "Experimental protocol"
    },
    {'aliases':[biosample_protocol],
     "protocol_type":"Cell culture protocol"
    }
    ]
)


{'publication': [], 'microscope_setting_a2': [{'tier_level': 'A2', 'aliases': ['thomas-gregor-lab:mic_set_BAF180']}], 'biosource': [], 'experiment_set_replicate': [], 'protocol': [], 'experiment_mic': [], 'imaging_path': [], 'file_processed': [], 'biosample': [], 'file_microscopy': []}


In [19]:
# disabled bcc

def add_to_dict(pre_alias, typ, main_dict, other_val):
    dict_to_add = {}
    dict_to_add['aliases'] = [pre_alias,]
    if other_val:
        for i in other_val:
            dict_to_add[i] = other_val[i]
    main_dict[typ].append(dict_to_add)
    return main_dict


import copy
TEMP = copy.deepcopy(TEMPO)

all_sets = []
for num, line in enumerate(sum_list): 
    #ADD SET
    if line['Experiment Sets (alias root)']:
        my_set = submitter_lab + line['Experiment Sets (alias root)']
        print(my_set)
        all_sets.append(my_set)
        TEMP = add_to_dict(my_set, 'experiment_set_replicate', TEMP, {'description':line['Set Description']})
    
        #ADD BIOSOUCE
        # does it once per set
        if not bio_source:
            my_biosoruce = submitter_lab + line['Experiment Sets (alias root)']
            TEMP = add_to_dict(my_biosource, 'biosource', TEMP, {})
        
        
    #ADD BCC
    if add_bcc:
        my_bcc = submitter_lab + 'bcc_' + line['Biosample (alias root)']
        TEMP = add_to_dict(my_bcc, 'biosample_cell_culture' , TEMP, {})
        
    #ADD BIOSAMPLE
    my_biosample = submitter_lab+'biosample_' + line['Biosample (alias root)']
    extra = {'biosource':bio_source, 'biosample_protocols':biosample_protocol}
    if add_bcc:
        extra['cell_culture_details'] = my_bcc
    TEMP = add_to_dict(my_biosample, 'biosample', TEMP, extra)
    
    
    #DEFINE EXPERIMENT
    my_exp = submitter_lab + line['Experiment (alias root)']
    
    # ADD FILES
    raw_files = []
    proc_files = []
    
    if line['Number of Raw Files']:
        for n in range(int(line['Number of Raw Files'])):
            raw_f_alias = my_exp + '_raw_file_' + str(n+1)
            raw_files.append(raw_f_alias)
            TEMP = add_to_dict(raw_f_alias, 'file_microscopy', TEMP, {})
  
    if line['Number of Processed Files']:
        for n in range(int(line['Number of Processed Files'])):
            extra = {'file_format': 'spt',
                    'file_type': 'spt results'}
            proc_f_alias = my_exp + '_proc_file_' + str(n+1)
            proc_files.append(proc_f_alias)
            TEMP = add_to_dict(proc_f_alias, 'file_processed', TEMP, extra)

    
    #ADD EXPERIMENT
    extra = {'replicate_set':my_set, 'protocol':experiment_protocol, 'biosample':my_biosample,
             'experiment_type': "SPT", "microscope_settings_master": setting_alias}
    if raw_files:
        extra['files'] = raw_files
    if proc_files:
        extra['processed_files'] = proc_files
    exp_pieces = my_exp.split('_')
    if exp_pieces[-1].startswith('B'):
        extra['bio_rep_no'] = exp_pieces[-1][1:]
        extra['tec_rep_no'] = '1'
    elif exp_pieces[-1].startswith('T'):
        extra['bio_rep_no'] = exp_pieces[-2][1:]
        extra['tec_rep_no'] = exp_pieces[-1][1:]
    TEMP = add_to_dict(my_exp, 'experiment_mic', TEMP, extra)
    
# ADD PUBLICATION
my_pub = submitter_lab + 'publication_' + project
extra = {'exp_sets_prod_in_pub': all_sets}
TEMP = add_to_dict(my_pub, 'publication', TEMP, extra)

# for i in TEMP:
#     print(i)
#     if TEMP[i]:
#         for x in TEMP[i]:
#             print(x)
#         print()
#     print()
#     print('--------------')
            
    

robert-coleman-lab:Halo_BAF180WT_SAHA
robert-coleman-lab:Halo_BAF180WT_DMSO
robert-coleman-lab:Halo_BAF180_BDdel_SAHA
robert-coleman-lab:Halo_BAF180_Bddel_DMSO
robert-coleman-lab:Halo_BAF180WT_SNAP_HP1a
robert-coleman-lab:Halo_BAF180WT_H3.3_SNAP
robert-coleman-lab:Halo_BAF180_Bddel_SNAP_HP1a
robert-coleman-lab:Halo_BAF180_Bddel_H3.3_SNAP


In [20]:
# duplicate mic to mic path
TEMP['experiment_mic_path'] = TEMP['experiment_mic']
schema_name = get_schema_names(my_key) 
schema_name['ExperimentMic_Path'] = 'experiment_mic_path'

In [21]:
append_items_to_xls(excel_file, TEMP, schema_name, comment = False)

new excel is stored as /Users/koray/Desktop/Wrangling/Labs/Coleman/160515_spt_template_with_items.xls


In [7]:
t = """robert-coleman-lab:Halo_BAF180WT_SAHA
robert-coleman-lab:Halo_BAF180WT_DMSO
robert-coleman-lab:Halo_BAF180_BDdel_SAHA
robert-coleman-lab:Halo_BAF180_Bddel_DMSO
robert-coleman-lab:Halo_BAF180WT_SNAP_HP1a
robert-coleman-lab:Halo_BAF180WT_H3.3_SNAP
robert-coleman-lab:Halo_BAF180_Bddel_SNAP_HP1a
robert-coleman-lab:Halo_BAF180_Bddel_H3.3_SNAP
robert-coleman-lab:Halo_BAF180_Bddel_H3.3_SNAP"""

print(', '.join(t.split('\n')))

robert-coleman-lab:Halo_BAF180WT_SAHA, robert-coleman-lab:Halo_BAF180WT_DMSO, robert-coleman-lab:Halo_BAF180_BDdel_SAHA, robert-coleman-lab:Halo_BAF180_Bddel_DMSO, robert-coleman-lab:Halo_BAF180WT_SNAP_HP1a, robert-coleman-lab:Halo_BAF180WT_H3.3_SNAP, robert-coleman-lab:Halo_BAF180_Bddel_SNAP_HP1a, robert-coleman-lab:Halo_BAF180_Bddel_H3.3_SNAP, robert-coleman-lab:Halo_BAF180_Bddel_H3.3_SNAP
